In [ ]:
!pip install transformers
!pip install torch
!pip install scikit-learn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
used = pd.read_csv("/content/drive/MyDrive/result_data.csv") #/content/drive/MyDrive/ml/finbert/res_data_all.csv
used.columns = ['name','v1','v2','real','before','year']

In [ ]:
#for the data after 2006 , volatility values are not logged in the later2007.csv file!!!!
used.loc[used['year']>2006,'real'] = used.loc[df4['year']>2006,'real'].apply(lambda x: math.log(x) )
used.loc[used['year']>2006,'before'] = used.loc[df4['year']>2006,'before'].apply(lambda x: math.log(x) )

In [ ]:
#NaN = np.nan
import numpy as np
used['features']=""
for i in range(len(used)):  
  count = 0
  my_list = used.loc[i,['v1']].v1.split()
  other_list = used.loc[i,['v2']].v2.split()
  if '[' in my_list:
    my_list.remove('[')
  if '[' in other_list:
    other_list.remove('[')
  if ']' in my_list:
    my_list.remove(']')
  if ']' in other_list:
    other_list.remove(']')
  for x in my_list:
    if my_list[count]!='[':
      if count==0:
        my_list[count]=my_list[count][1:]
        other_list[count]=other_list[count][1:]
      if count==len(my_list)-1:
        my_list[count]=my_list[count][:-1]
        other_list[count]=other_list[count][:-1]
      my_list[count] = float(my_list[count])
      other_list[count] = float(other_list[count])
      count+=1
      
  my_list=np.array(my_list)
  other_list=np.array(other_list)
  overall =np.hstack((my_list,other_list)) 
  #overall =np.hstack((my_list,other_list,used.loc[i,['before']].before)) --> if previous value will be added for the kogan levin data

  used.at[i,'features']= overall

# tf and tf-idf features

In [ ]:
df2 =used

tf features:

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
  
# Create a Vectorizer Object
tf_idf_vec_smooth = TfidfVectorizer(use_idf=False,  
                        #smooth_idf=True,  
                        ngram_range=(1,1),stop_words='english',max_features=2000)
 
 
tf_idf_data = tf_idf_vec_smooth.fit_transform(df2['Text'].to_numpy())

In [ ]:
df2['tf'] = tf_idf_data.toarray().tolist()#arr.toarray().tolist()

tf-idf features

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
  
#with smooth
tf_idf_vec_smooth = TfidfVectorizer(use_idf=True,  
                        smooth_idf=True,  
                        ngram_range=(1,1),stop_words='english',max_features=2000)
 
 
tf_idf_data = tf_idf_vec_smooth.fit_transform(df2['Text'].to_numpy())


In [ ]:
df2['tfidf'] = tf_idf_data.toarray().tolist()#arr.toarray().tolist()

whether adding previous volatility values or not:

In [ ]:
import numpy as np
df2['features']=""
for i in range(len(df2)):  
  count = 0
  my_list = df2.loc[i,['tf']].tf.split(", ") #df2.loc[i,['tfidf']].tfidf.split(", ") 
  if '[' in my_list:
    my_list.remove('[')
  if ']' in my_list:
    my_list.remove(']')
  for x in my_list:
    if my_list[count]!='[':
      if count==0:
        my_list[count]=my_list[count][1:]
      if count==len(my_list)-1:
        my_list[count]=my_list[count][:-1]
      my_list[count] = float(my_list[count])
      count+=1
      
  my_list=np.array(my_list)
  overall =np.hstack((my_list,df3.loc[i,['before']].before))
  df2.at[i,'features']= overall#my_list

relevant year interval arrangement is made for the code below

In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np
X_train = np.vstack(df2[df2['year'].isin([2002,2003,2001,2000,2004])]['features'].to_numpy()) #.isin([1998,1997,1996,2000,1999])]['features'].to_numpy())
y_train = df2[df2['year'].isin([2002,2003,2001,2000,2004])]['real'].to_numpy()
X_test  = np.vstack(df2[df2['year']==2005]['features'].to_numpy())
y_test = df2[df2['year']==2005]['real'].to_numpy()

In [ ]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np
regr = make_pipeline(StandardScaler(), SVR())#(C=1.0, epsilon=0.2))
regr.fit(X_train,y_train)
y_pred= regr.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test,y_pred)

# bert-based embeddings

relevant year interval arrangement is made for the code below

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
df3 = used_df
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np
for i in range(2002,2017):
	print(i,'-',i+4,'\n')
	X_train = np.vstack(df3[df3['year'].isin([i,i+1,i+2,i+3,i+4])]['features'].to_numpy()) #.isin([1998,1997,1996,2000,1999])]['features'].to_numpy())
	y_train = df3[df3['year'].isin([i,i+1,i+2,i+3,i+4])]['real'].to_numpy()
	X_test  = np.vstack(df3[df3['year']==i+5]['features'].to_numpy())
	y_test = df3[df3['year']==i+5]['real'].to_numpy()
	#RANDOM FOREST  
	print('random forest\n')
	rf = RandomForestRegressor()
	rf.fit(X_train,y_train)
	y_pred= rf.predict(X_test)
	rs = mean_squared_error(y_test,y_pred)
	print('result: ',rs,'\n' )
	
	
	print('XGBRegressor\n')
	model = XGBRegressor(n_estimators=100,objective='reg:squarederror')
	model.fit(X_train, y_train)
	y_pred = model.predict(X_test)
	rs = mean_squared_error(y_test,y_pred)
	print('result: ',rs,'\n' )
	
	print('SVR \n')
	regr = make_pipeline(StandardScaler(), SVR())#(C=1.0, epsilon=0.2))
	regr.fit(X_train,y_train)
	y_pred= regr.predict(X_test)
	rs = mean_squared_error(y_test,y_pred)
	print('result: ',rs,'\n' )